In [1]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import utils

In [2]:
train_data = pd.read_csv('./temperature-only.csv')['TempAvgF']
train_data = (train_data - train_data.mean())/train_data.std()
train_data = train_data.values[:-1]
train_data.shape

(1318,)

In [3]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence,self).__init__()
        self.lstm1 = nn.LSTM(1,16, 2)
        self.lstm2 = nn.LSTM(16,1)
 
    def forward(self,seq, hc = None):
        out = []
        if hc == None:
            hc1, hc2 = None, None
        else:
            hc1, hc2 = hc
        
        for X in seq.chunk(seq.size(1),dim=0):
            tmp, hc1 = self.lstm1(X, hc1)
            X_in, hc2 = self.lstm2(tmp, hc2)
            out.append(X_in)
        return torch.stack(out).squeeze(1),(hc1,hc2)

In [4]:
seq = Sequence()
criterion = nn.MSELoss()
optimizer = optim.Adam(seq.parameters(), lr=0.005)

In [5]:
for epoch in range(50):
    total_loss = 0
    running_error = 0
    passes = 0
    for i in range(0, 1318, 5):
        data = train_data[i:i+32]
        xs = torch.Tensor(data[:-1]).view(-1,1,1)
        ys = torch.Tensor(data[1:])
        
        lstm_out, _= seq(xs)
        loss = criterion(lstm_out.view(-1), ys)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        passes += 1
        with torch.no_grad():
            total_loss += loss.data[0]
            error = utils.get_error(lstm_out.view(1, -1), ys.view(1, -1))
            running_error += error 
        
    print("epoch {}, loss {:2.1%}, error {:2.1%}".format(epoch, total_loss/passes, running_error/passes))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch 0, loss 78.1%, error 61.3%
epoch 1, loss 32.4%, error 31.4%
epoch 2, loss 24.7%, error 24.9%
epoch 3, loss 23.5%, error 23.5%
epoch 4, loss 22.8%, error 22.9%
epoch 5, loss 22.7%, error 23.1%
epoch 6, loss 22.5%, error 22.9%
epoch 7, loss 22.1%, error 22.3%
epoch 8, loss 21.9%, error 22.1%
epoch 9, loss 21.9%, error 21.8%
epoch 10, loss 21.9%, error 21.9%
epoch 11, loss 21.7%, error 21.7%
epoch 12, loss 21.3%, error 21.6%
epoch 13, loss 21.1%, error 21.2%
epoch 14, loss 21.1%, error 20.7%
epoch 15, loss 20.4%, error 20.0%
epoch 16, loss 20.4%, error 19.8%
epoch 17, loss 20.6%, error 20.0%
epoch 18, loss 20.2%, error 19.9%
epoch 19, loss 19.7%, error 18.7%
epoch 20, loss 19.8%, error 19.0%
epoch 21, loss 19.4%, error 18.5%
epoch 22, loss 19.5%, error 18.8%
epoch 23, loss 20.1%, error 19.1%
epoch 24, loss 19.4%, error 18.1%
epoch 25, loss 19.3%, error 18.2%
epoch 26, loss 19.4%, error 18.4%
epoch 27, loss 19.2%, error 18.3%
epoch 28, loss 18.5%, error 17.0%
epoch 29, loss 18.7%, er

In [6]:
def predict_all(n):
    data = train_data[:n]
    xs = torch.Tensor(data[:-1]).view(-1,1,1)
    ys = torch.Tensor(data[1:])
    predictions, _ = seq(xs)
    return predictions.view(-1).detach().numpy(), ys.view(-1).detach().numpy()

pred, actual = predict_all(1318)
p = pd.Series(pred.T)
a = pd.Series(actual.T)
df = pd.DataFrame()
df['Predictions'] = p
df['Actual'] = a
df.head()
df.to_csv('./compared.csv')